# Formation South Green 2021  

##  Initiation à l’analyse de données Minion

### PART 2

Created by J.Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and G. Sarah (AGAP-CIRAD)

Septembre 2021


# 1. Assemblies

More contiguous genome assemblies can be generated using long sequencing read but assembly is not a quite pace. Eukaryotic genomes assembly is a complex task (large genome sizes, high rates of repeated sequences, high heterozygosity levels and even polyploidy), and while prokaryotic genomes may appear less challenging, specific features such as circular DNA molecules, must be taken into consideration to achieve high quality assembly.

* For assembly, ONT recommend sequencing a human genome to a minimum depth of 30x of 25–35 kb reads.
However, sequencing to a depth of 60x is advisable to obtain the best assembly metrics. We also recommend basecalling in high accuracy mode. Greatest contig N50 is usually obtained with Shasta and Flye. Polishing/Correction is also recomendated (Racon and Medaka). https://nanoporetech.com/sites/default/files/s3/literature/human-genome-assembly-workflow.pdf


* Long reads simplify genome assembly, with the ability to span repeat-rich sequences (characteristic of  antimicrobial resistance genes) and structural variants. Nanopore sequencing also shows a lack of
bias in GC-rich regions, in contrast to other sequencing platforms. To perform microbial genome assembly, we suggest using the third-party de novo assembly tool Flye. We also recommend one round of polishing with Medaka. https://nanoporetech.com/sites/default/files/s3/literature/microbial-genome-assembly-workflow.pdf 

Flye  https://github.com/fenderglass/Flye 

Canu  https://canu.readthedocs.io/en/latest/quick-start.html

Miniasm  https://github.com/lh3/miniasm + Minipolish version https://github.com/rrwick/Minipolish

Shasta  https://github.com/chanzuckerberg/shasta

Smartdenovo  https://github.com/ruanjue/smartdenovo

Raven  https://github.com/lbcb-sci/raven

### 1.1 Assembly using Flye

We are going to assembly some Clones by using Flye https://github.com/fenderglass/Flye

Flye generates the concatenation of multiple disjoint genomic segments called disjointigs to build a repeat graph. Reads are mapped to this repeat graph to resolve conflicts (unbridged repeats) and output contigs.

In [2]:
# declare your clone number
CLONE="Clone20"
# create a repertory for save flye assembly results
cd ~/SG-ONT-2021/RESULTS/
pwd

/home/jovyan/SG-ONT-2021


In [3]:
# Run flye
conda activate flye
time flye --nano-raw DATA/${CLONE}/ONT/${CLONE}.fastq.gz --genome-size 1000000 --out-dir FLYE --threads 4
conda deactivate

(flye) [2021-09-14 21:46:04] INFO: Starting Flye 2.9-b1768
[2021-09-14 21:46:04] INFO: >>>STAGE: configure
[2021-09-14 21:46:04] INFO: Configuring run
[2021-09-14 21:46:08] INFO: Total read length: 160316874
[2021-09-14 21:46:08] INFO: Input genome size: 1000000
[2021-09-14 21:46:08] INFO: Estimated coverage: 160
[2021-09-14 21:46:08] INFO: Reads N50/N90: 22312 / 7619
[2021-09-14 21:46:08] INFO: Minimum overlap set to 8000
[2021-09-14 21:46:08] INFO: >>>STAGE: assembly
[2021-09-14 21:46:08] INFO: Assembling disjointigs
[2021-09-14 21:46:08] INFO: Reading sequences
[2021-09-14 21:46:21] INFO: Counting k-mers:
0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 
[2021-09-14 21:47:29] INFO: Filling index table (1/2)
0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 
[2021-09-14 21:47:54] INFO: Filling index table (2/2)
0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 
[2021-09-14 21:49:05] INFO: Extending reads
[2021-09-14 21:52:03] INFO: Overlap-based coverage: 123
[2021-09-14 21:52:03] INFO: Median overla

: 1

### 1.2 Assembly using Raven

In [8]:
# create a repertory for save raven assembly results
mkdir ~/SG-ONT-2021/RESULTS/RAVEN
cd ~/SG-ONT-2021/RESULTS/RAVEN
pwd

(base) (base) (base) /home/jovyan/SG-ONT-2021/RESULTS/RAVEN
(base) 

: 1

In [9]:
# Run raven
conda activate raven
time raven -p 0 --graphical-fragment-assembly ${CLONE}_raven.gfa -t 4 ../../DATA/${CLONE}/ONT/${CLONE}.fastq.gz > ${CLONE}_raven.fasta
conda deactivate

(base) (raven) [raven::] loaded 10804 sequences 3.536120s
[raven::Graph::Construct] minimized 0 - 10804 / 10804 3.162863s
[raven::Graph::Construct] mapped sequences 12.824156s
[raven::Graph::Construct] annotated piles 0.084350s
[raven::Graph::Construct] removed contained sequences 0.021729s
[raven::Graph::Construct] removed chimeric sequences 0.013587s
[raven::Graph::Construct] reached checkpoint 0.103159s
[raven::Graph::Construct] minimized 0 - 198 / 198 0.894041s
[raven::Graph::Construct] mapped valid sequences 0.720659s
[raven::Graph::Construct] updated overlaps 0.000669s
[raven::Graph::Construct] removed false overlaps 0.009050s
[raven::Graph::Construct] stored 396 nodes 0.082866s
[raven::Graph::Construct] stored 2708 edges 0.001006s
[raven::Graph::Construct] reached checkpoint 0.179135s
[raven::Graph::Construct] 18.097270s
[raven::Graph::Assemble] removed transitive edges 0.003050s
[raven::Graph::Assemble] reached checkpoint 0.083759s
[raven::Graph::Assemble] removed tips and bubb

: 1

* How many contigs were obtained by Flye and Raven ? Please fill in results in the shared [file](https://lite.framacalc.org/9pd3-ont_sg_2021)
* Calculate first statistics about assemblies: N50 and lenght contig mean.

In [10]:
grep ^'>' ~/SG-ONT-2021/RESULTS/*/*.fasta

/home/jovyan/SG-ONT-2021/RESULTS/FLYE/assembly.fasta:>contig_1
/home/jovyan/SG-ONT-2021/RESULTS/FLYE/assembly.fasta:>contig_2
/home/jovyan/SG-ONT-2021/RESULTS/FLYE/assembly.fasta:>contig_3
/home/jovyan/SG-ONT-2021/RESULTS/RAVEN/Clone20_raven.fasta:>Utg398 LN:i:728202 RC:i:120 XO:i:0
/home/jovyan/SG-ONT-2021/RESULTS/RAVEN/Clone20_raven.fasta:>Utg400 LN:i:390114 RC:i:64 XO:i:0
/home/jovyan/SG-ONT-2021/RESULTS/RAVEN/Clone20_raven.fasta:>Utg402 LN:i:49172 RC:i:6 XO:i:0
/home/jovyan/SG-ONT-2021/RESULTS/RAVEN/Clone20_raven.fasta:>Utg404 LN:i:49281 RC:i:7 XO:i:0
(base) 

: 1

# 2. Polishing assemblies with Racon

Racon corrects raw contigs generated by rapid assembly methods with original ONT reads.

From 2 to 4 racon rounds are usually used by the community. 

Polish contigs assembled by Flye using two rounds of racon !.

In [16]:
CLONE=Clone20

(medaka) 

: 1

In [11]:
conda activate racon
racon

(racon) [racon::] error: missing input file(s)!
usage: racon [options ...] <sequences> <overlaps> <target sequences>

    #default output is stdout
    <sequences>
        input file in FASTA/FASTQ format (can be compressed with gzip)
        containing sequences used for correction
    <overlaps>
        input file in MHAP/PAF/SAM format (can be compressed with gzip)
        containing overlaps between sequences and target sequences
    <target sequences>
        input file in FASTA/FASTQ format (can be compressed with gzip)
        containing sequences which will be corrected

    options:
        -u, --include-unpolished
            output unpolished target sequences
        -f, --fragment-correction
            perform fragment correction instead of contig polishing
            (overlaps file should contain dual/self overlaps!)
        -w, --window-length <int>
            default: 500
            size of window on which POA is performed
        -q, --quality-threshold <float>
    

: 1

## Racon first round

In [12]:

mkdir -p ~/SG-ONT-2021/RESULTS/FLYE_RACON
cd ~/SG-ONT-2021/RESULTS/FLYE_RACON
time minimap2 -t 4 ../FLYE/assembly.fasta ../../DATA/${CLONE}/ONT/${CLONE}.fastq.gz 1> assembly.minimap4racon1.paf
time racon -t 4 ../../DATA/${CLONE}/ONT/${CLONE}.fastq.gz assembly.minimap4racon1.paf ../FLYE/assembly.fasta > assembly.racon1.fasta

(racon) (racon) (racon) (racon) [M::mm_idx_gen::0.121*0.82] collected minimizers
[M::mm_idx_gen::0.167*0.87] sorted minimizers
[M::main::0.167*0.87] loaded/built the index for 3 target sequence(s)
[M::mm_mapopt_update::0.175*0.87] mid_occ = 6
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 3
[M::mm_idx_stat::0.180*0.87] distinct minimizers: 214643 (97.74% are singletons); average occurrences: 1.031; average spacing: 5.355
[M::worker_pipeline::15.767*1.00] mapped 10804 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -t 1 ../FLYE/assembly.fasta ../../DATA/Clone20/ONT/Clone20.fastq.gz
[M::main] Real time: 15.775 sec; CPU: 15.744 sec; Peak RSS: 0.171 GB

real	0m15.957s
user	0m15.412s
sys	0m0.352s
(racon) [racon::Polisher::initialize] loaded target sequences 0.019625 s
[racon::Polisher::initialize] loaded sequences 3.955057 s
[racon::Polisher::initialize] loaded overlaps 0.019120 s
[racon::Polisher::initialize] aligning overlaps [====================] 125.001303 s


: 1

## Racon second round

In [13]:
# second round
cd ~/SG-ONT-2021/RESULTS/FLYE_RACON
time minimap2 -t 4 assembly.racon1.fasta ../../DATA/${CLONE}/ONT/${CLONE}.fastq.gz 1> assembly.minimap4racon2.paf
time racon -t 4 ../../DATA/${CLONE}/ONT/${CLONE}.fastq.gz assembly.minimap4racon2.paf assembly.racon1.fasta > assembly.racon2.fasta
conda deactivate

(racon) [M::mm_idx_gen::0.093*1.03] collected minimizers
[M::mm_idx_gen::0.139*1.02] sorted minimizers
[M::main::0.139*1.02] loaded/built the index for 3 target sequence(s)
[M::mm_mapopt_update::0.147*1.02] mid_occ = 6
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 3
[M::mm_idx_stat::0.152*1.01] distinct minimizers: 213223 (97.67% are singletons); average occurrences: 1.032; average spacing: 5.357
[M::worker_pipeline::15.971*1.00] mapped 10804 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -t 1 assembly.racon1.fasta ../../DATA/Clone20/ONT/Clone20.fastq.gz
[M::main] Real time: 15.979 sec; CPU: 15.967 sec; Peak RSS: 0.170 GB

real	0m16.005s
user	0m15.568s
sys	0m0.421s
(racon) [racon::Polisher::initialize] loaded target sequences 0.016481 s
[racon::Polisher::initialize] loaded sequences 3.885330 s
[racon::Polisher::initialize] loaded overlaps 0.018785 s
[racon::Polisher::initialize] aligning overlaps [====================] 131.590772 s
[racon::Polisher::initial

: 1

# 3. Assemblies correction with Medaka

Correction can improve the consensus sequence for a draft genome assembly.

Medaka uses fast5 files to correct contigs using trained models. These models are free available.

Medaka allows you train a model from using fast5 of your favorite specie. You can use it directly to obtain a consensus from you favorite organism.

## 1.3 Correct assemblies with Medaka

### 1.3.1 Before to correct assemblies, index them

In [42]:
conda activate medaka
cd ~/SG-ONT-2021/RESULTS/FLYE_RACON

(medaka) (medaka) 

: 1

In [8]:
time samtools faidx assembly.racon2.fasta 
time minimap2 -d assembly.racon2.fasta.mmi assembly.racon2.fasta

(medaka) 
real	0m0.021s
user	0m0.008s
sys	0m0.009s
(medaka) [M::mm_idx_gen::0.093*1.04] collected minimizers
[M::mm_idx_gen::0.140*1.66] sorted minimizers
[M::main::0.181*1.49] loaded/built the index for 3 target sequence(s)
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 3
[M::mm_idx_stat::0.193*1.42] distinct minimizers: 213101 (97.66% are singletons); average occurrences: 1.032; average spacing: 5.358; total length: 1178330
[M::main] Version: 2.22-r1101
[M::main] CMD: minimap2 -d assembly.racon2.fasta.mmi assembly.racon2.fasta
[M::main] Real time: 0.204 sec; CPU: 0.281 sec; Peak RSS: 0.015 GB

real	0m0.209s
user	0m0.210s
sys	0m0.074s
(medaka) 

: 1

In [9]:
#create a medaka repertory
mkdir -p ~/SG-ONT-2021/RESULTS/FLYE_RACON_MEDAKA
cd ~/SG-ONT-2021/RESULTS/FLYE_RACON_MEDAKA

(medaka) (medaka) (medaka) 

: 1

### 1.3.2 Medaka_consensus

Medaka is a tool to create a consensus sequence of nanopore sequencing data. 

This task is performed using neural networks applied a pileup of individual sequencing reads against a draft assembly.

It outperforms graph-based methods operating on basecalled data, and can be competitive with state-of-the-art signal-based methods whilst being much faster.

As input medaka accepts reads in either a .fasta or a .fastq file. It requires a draft assembly as a .fasta.

### Check the usage of medaka_consensus

In [43]:
medaka_consensus


medaka 1.2.2
------------

Assembly polishing via neural networks. The input assembly should be
preprocessed with racon.

medaka_consensus [-h] -i <fastx>

    -h  show this help text.
    -i  fastx input basecalls (required).
    -d  fasta input assembly (required).
    -o  output folder (default: medaka).
    -g  don't fill gaps in consensus with draft sequence.
    -m  medaka model, (default: r941_min_high_g360).
        Available: r103_min_high_g345, r103_min_high_g360, r103_prom_high_g360, r103_prom_snp_g3210, r103_prom_variant_g3210, r10_min_high_g303, r10_min_high_g340, r941_min_fast_g303, r941_min_high_g303, r941_min_high_g330, r941_min_high_g340_rle, r941_min_high_g344, r941_min_high_g351, r941_min_high_g360, r941_prom_fast_g303, r941_prom_high_g303, r941_prom_high_g330, r941_prom_high_g344, r941_prom_high_g360, r941_prom_high_g4011, r941_prom_snp_g303, r941_prom_snp_g322, r941_prom_snp_g360, r941_prom_variant_g303, r941_prom_variant_g322, r941_prom_variant_g360.
        Alte

: 1

### Check the medaka model to use

Medaka models are named to indicate i) the pore type, ii) the sequencing device (MinION or PromethION), iii) the basecaller variant, and  iv) the basecaller version

{pore}_{device}_{caller variant}_{caller version}

examples: 

r941_min_fast_g303 : MiniON R9.4.1 flowcells using the fast Guppy basecaller version 3.0.3. 


In [44]:
medaka tools list_models

Available: r103_min_high_g345, r103_min_high_g360, r103_prom_high_g360, r103_prom_snp_g3210, r103_prom_variant_g3210, r10_min_high_g303, r10_min_high_g340, r941_min_fast_g303, r941_min_high_g303, r941_min_high_g330, r941_min_high_g340_rle, r941_min_high_g344, r941_min_high_g351, r941_min_high_g360, r941_prom_fast_g303, r941_prom_high_g303, r941_prom_high_g330, r941_prom_high_g344, r941_prom_high_g360, r941_prom_high_g4011, r941_prom_snp_g303, r941_prom_snp_g322, r941_prom_snp_g360, r941_prom_variant_g303, r941_prom_variant_g322, r941_prom_variant_g360
Default consensus:  r941_min_high_g360
Default snp:  r941_prom_snp_g360
Default variant:  r941_prom_variant_g360
(medaka) 

: 1

### Check the medaka model to use

In [46]:
# go to medaka repertory results
cd ~/SG-ONT-2021/RESULTS/FLYE_RACON_MEDAKA
#rm -rf ~/SG-ONT-2021/RESULTS/FLYE_RACON_MEDAKA/MEDAKA_CONSENSUS

(medaka) (medaka) (medaka) 

: 1

### Medaka in FLYE + RACON

We run medaka on the FLYE + polished with racon assembly.

In [41]:
# medaka_consensus -i ${BASECALLS} -d ${DRAFT} -o ${OUTDIR} -t ${NPROC} -m r941_min_high_g360
time medaka_consensus -i /home/jovyan/SG-ONT-2021/DATA/Clone20/ONT/Clone20.fastq.gz -d ~/SG-ONT-2021/RESULTS/FLYE_RACON/assembly.racon2.fasta -o MEDAKA_CONSENSUS -t 4 -m r941_min_high_g360

(medaka) Checking program versions
This is medaka 1.2.2
Program    Version    Required   Pass     
bcftools   1.11       1.9        True     
bgzip      1.11       1.9        True     
minimap2   2.22       2.11       True     
samtools   1.11       1.9        True     
tabix      1.11       1.9        True     
Aligning basecalls to draft
Removing previous index file /home/jovyan/SG-ONT-2021/RESULTS/FLYE_RACON/assembly.racon2.fasta.mmi
Removing previous index file /home/jovyan/SG-ONT-2021/RESULTS/FLYE_RACON/assembly.racon2.fasta.fai
Constructing minimap index.
[M::mm_idx_gen::0.102*1.01] collected minimizers
[M::mm_idx_gen::0.142*1.45] sorted minimizers
[M::main::0.160*1.40] loaded/built the index for 3 target sequence(s)
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 3
[M::mm_idx_stat::0.164*1.39] distinct minimizers: 213101 (97.66% are singletons); average occurrences: 1.032; average spacing: 5.358; total length: 1178330
[M::main] Version: 2.22-r1101
[M::main] CMD: minim

: 1


## Improve parallelisme to large datasets at scale

The medaka_consensus program is good for simple datasets but perhaps not optimal for running large datasets at scale. 

Check complemental documentation [here](https://nanoporetech.github.io/medaka/installation.html#improving-parallelism)

Three steps are need to performed at scale :

```
# align reads to assembly
mini_align -i basecalls.fasta -r assembly.fasta -P -m \
    -p calls_to_draft.bam -t <threads>

# run lots of jobs like this, change model as appropriate
mkdir results
medaka consensus calls_to_draft.bam results/contigs1-4.hdf \
    --model r941_min_fast_g303 --batch 200 --threads 8 \
    --region contig1 contig2 contig3 contig4
...

# wait for jobs, then collate results
medaka stitch results/*.hdf polished.assembly.fasta
```
Note : medaka_consensus is not medaka consensus !

### Conclusion

Here, we have obtained an pseudomolecule of a clone by using Flye, we have polished it twice with racon and corrected it with medaka models.

You can do similar pipeline using the RAVEN assembler with your favorite Clone.

We will to compare results in next practical "Quality Assemblies". 